In [153]:
# client.py
import json
import backoff
import requests
from requests.exceptions import ConnectionError
from singer import metrics, utils
import singer

LOGGER = singer.get_logger()


class Server5xxError(Exception):
    pass


class Server429Error(Exception):
    pass


class MambuError(Exception):
    pass


class MambuBadRequestError(MambuError):
    pass


class MambuUnauthorizedError(MambuError):
    pass


class MambuRequestFailedError(MambuError):
    pass


class MambuNotFoundError(MambuError):
    pass


class MambuMethodNotAllowedError(MambuError):
    pass


class MambuConflictError(MambuError):
    pass


class MambuForbiddenError(MambuError):
    pass


class MambuUnprocessableEntityError(MambuError):
    pass


class MambuInternalServiceError(MambuError):
    pass


ERROR_CODE_EXCEPTION_MAPPING = {
    400: MambuBadRequestError,
    401: MambuUnauthorizedError,
    402: MambuRequestFailedError,
    403: MambuForbiddenError,
    404: MambuNotFoundError,
    405: MambuMethodNotAllowedError,
    409: MambuConflictError,
    422: MambuUnprocessableEntityError,
    500: MambuInternalServiceError}


def get_exception_for_error_code(error_code):
    return ERROR_CODE_EXCEPTION_MAPPING.get(error_code, MambuError)

def raise_for_error(response):
    try:
        response.raise_for_status()
    except (requests.HTTPError, requests.ConnectionError) as error:
        try:
            content_length = len(response.content)
            if content_length == 0:
                # There is nothing we can do here since Mambu has neither sent
                # us a 2xx response nor a response content.
                return
            response = response.json()
            if ('error' in response) or ('errorCode' in response):
                message = '%s: %s' % (response.get('error', str(error)),
                                      response.get('message', 'Unknown Error'))
                error_code = response.get('status')
                ex = get_exception_for_error_code(error_code)
                raise ex(message)
            else:
                raise MambuError(error)
        except (ValueError, TypeError):
            raise MambuError(error)


class MambuClient(object):
    def __init__(self,
                 username,
                 password,
                 base_url,
                 user_agent=None):
        self.__username = username
        self.__password = password
        self.base_url = base_url
        self.__user_agent = user_agent
        self.__session = requests.Session()
        self.__verified = False

    def __enter__(self):
        self.__verified = self.check_access()
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        self.__session.close()

    @backoff.on_exception(backoff.expo,
                          Server5xxError,
                          max_tries=5,
                          factor=2)
    def check_access(self):
        if self.__username is None or self.__password is None:
            raise Exception('Error: Missing username or password in config.json.')
        if self.base_url is None:
            raise Exception('Error: Missing base_url in cofig.json.')
        headers = {}
        # Endpoint: simple API call to return a single record (org settings) to test access
        # https://support.mambu.com/docs/organisational-settings-api#get-organisational-settings
        endpoint = 'settings/organization'
        url = '{}/{}'.format(self.base_url, endpoint)
        if self.__user_agent:
            headers['User-Agent'] = self.__user_agent
        headers['Accept'] = 'application/vnd.mambu.v1+json'
        response = self.__session.get(
            url=url,
            headers=headers,
            # Basic Authentication: https://api.mambu.com/?http#authentication
            auth=(self.__username, self.__password))
        if response.status_code != 200:
            LOGGER.error('Error status_code = {}'.format(response.status_code))
            raise_for_error(response)
        else:
            return True


    @backoff.on_exception(backoff.expo,
                          (Server5xxError, ConnectionError, Server429Error),
                          max_tries=5,
                          factor=2)
    def request(self, method, path=None, url=None, json=None, version='v2', **kwargs):
        if not self.__verified:
            self.__verified = self.check_access()

        if not url and path:
            url = '{}/{}'.format(self.base_url, path)

        if 'endpoint' in kwargs:
            endpoint = kwargs['endpoint']
            del kwargs['endpoint']
        else:
            endpoint = None

        if 'headers' not in kwargs:
            kwargs['headers'] = {}
        
        # Version represents API version (e.g. v2): https://api.mambu.com/?http#versioning
        kwargs['headers']['Accept'] = 'application/vnd.mambu.{}+json'.format(version)

        if self.__user_agent:
            kwargs['headers']['User-Agent'] = self.__user_agent

        if method == 'POST':
            kwargs['headers']['Content-Type'] = 'application/json'

        with metrics.http_request_timer(endpoint) as timer:
            response = self.__session.request(
                method=method,
                url=url,
                # Basic Authentication: https://api.mambu.com/?http#authentication
                auth=(self.__username, self.__password),
                json=json,
                **kwargs)
            timer.tags[metrics.Tag.http_status_code] = response.status_code

        if response.status_code >= 500:
            raise Server5xxError()

        if response.status_code != 200:
            raise_for_error(response)

        # paginationDetails=ON returns items-total as a header parameter in the response headers
        # Pagination: https://api.mambu.com/?http#pagination
        total_records = None
        if kwargs:
            if 'paginationDetails=ON' in kwargs['params']:
                total_records = int(response.headers.get('items-total', 0))
        return response.json(), total_records

In [148]:
# TESTING: Set working directory and load config file from working dir (where config/state file are)
import os
import json
import pprint

os.chdir('C:/Files/Work/Clients/Stitch/tap-mambu/tap_mambu')
my_config_path = 'tap_config.json'
with open(my_config_path) as file:
    CONFIG = json.load(file)

# my_state_path = 'state.json'
# with open(my_state_path) as file:
#     STATE = json.load(file)

# my_catalog_path = 'catalog.json'
# with open(my_catalog_path) as file:
#     CATALOG = json.load(file)

In [72]:
CONFIG

{'base_url': 'https://stitch.sandbox.mambu.com/api',
 'password': 'complicatedPassword1',
 'start_date': '2019-01-01T00:00:00Z',
 'user_agent': 'tap-mambu <jeff.huth@ybytecode.io>',
 'username': 'stitchAdmin'}

In [105]:
# TESTING: Test client.py get request with client
client = MambuClient(CONFIG['username'],
                     CONFIG['password'],
                     CONFIG['base_url'],
                     CONFIG['user_agent'])
params = {
    "detailsLevel": "FULL", 
    "sortBy": "lastModifiedDate:DESC",
    "offset": 0,
    "limit": 10,
    "paginationDetails": "ON"
}
endpoint = "clients"
method = "GET"
querystring = '&'.join(['%s=%s' % (key, value) for (key, value) in params.items()])

data, total_records = client.request(
    method=method,
    path=endpoint,
    params=querystring,
    endpoint=endpoint)

INFO METRIC: {"tags": {"status": "succeeded", "http_status_code": 200, "endpoint": "clients"}, "type": "timer", "metric": "http_request_duration", "value": 0.18400073051452637}


In [106]:
data

[{'_Custom_Fields_Clients': {'Position_Clients': 'Agricultural product sorter 3'},
  '_Food_Clients': {'Type_Clients': 'Chicken'},
  'activationDate': '2019-07-17T13:51:28-07:00',
  'addresses': [{'city': 'Marki',
    'country': '',
    'encodedKey': '8a818ed76bb7e868016bb867d6d718d6',
    'indexInList': 0,
    'line1': '34 Mihalakopoulou Avenue',
    'line2': '',
    'parentKey': '8a818ed76bb7e868016bb867d6011872',
    'postcode': '2647',
    'region': ''}],
  'assignedBranchKey': '8a818ed76bb7e868016bb8443c6313cb',
  'birthDate': '1963-05-21',
  'clientRoleKey': '8a818f9f6bb76211016bb83eb1310d6d',
  'creationDate': '2019-07-03T08:15:41-07:00',
  'encodedKey': '8a818ed76bb7e868016bb867d6011872',
  'firstName': 'Ela',
  'gender': 'FEMALE',
  'groupLoanCycle': 0,
  'homePhone': '+14437561558',
  'id': '0',
  'idDocuments': [],
  'lastModifiedDate': '2019-07-17T18:13:31-07:00',
  'lastName': 'Novaković',
  'loanCycle': 0,
  'mobilePhone': '+14437561558',
  'notes': 'Her business is the o

In [112]:
total_records

10

In [139]:
# transform.py

import re

# Convert camelCase to snake_case
def convert(name):
    regsub = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', regsub).lower()


# Convert keys in json array
def convert_array(arr):
    new_arr = []
    for i in arr:
        if isinstance(i, list):
            new_arr.append(convert_array(i))
        elif isinstance(i, dict):
            new_arr.append(convert_json(i))
        else:
            new_arr.append(i)
    return new_arr


# Convert keys in json
def convert_json(this_json):
    out = {}
    for key in this_json:
        new_key = convert(key)
        if isinstance(this_json[key], dict):
            out[new_key] = convert_json(this_json[key])
        elif isinstance(this_json[key], list):
            out[new_key] = convert_array(this_json[key])
        else:
            out[new_key] = this_json[key]
    return out


def remove_custom_nodes(this_json):
    if not isinstance(this_json, (dict, list)):
        return this_json
    if isinstance(this_json, list):
        return [remove_custom_nodes(vv) for vv in this_json]
    return {kk: remove_custom_nodes(vv) for kk, vv in this_json.items()
            if not(kk[:1] == '_')}



# Convert custom fields and sets
def convert_custom_fields(this_json):    
    new_json = this_json
    i = 0
    for record in this_json:
        cust_field_sets = []
        for key in record:
            if isinstance(record[key], dict):
                if key[:1] == '_':
                    cust_field_set = {}
                    cust_field_set['customFieldSetId'] = key
                    cust_field_set_fields = []
                    for cf_key, cf_value in record[key].items():
                        field = {}
                        field['customFieldId'] = cf_key
                        field['customFieldValue'] = cf_value
                        cust_field_set_fields.append(field)
                    cust_field_set['customFieldValues'] = cust_field_set_fields
                    cust_field_sets.append(cust_field_set)
        new_json[i]['customFieldSets'] = cust_field_sets
        i = i + 1
    return new_json


# Run all transforms: denests _embedded, removes _embedded/_links, and
#  converst camelCase to snake_case for fieldname keys.
def transform_json(this_json, path):
    new_json = remove_custom_nodes(convert_custom_fields(this_json))
    out = {}
    out[path] = new_json
    transformed_json = convert_json(out)
    return transformed_json[path]


In [137]:
transformed_data = transform_json(data, endpoint)

In [138]:
transformed_data

[{'activation_date': '2019-07-17T13:51:28-07:00',
  'addresses': [{'city': 'Marki',
    'country': '',
    'encoded_key': '8a818ed76bb7e868016bb867d6d718d6',
    'index_in_list': 0,
    'line1': '34 Mihalakopoulou Avenue',
    'line2': '',
    'parent_key': '8a818ed76bb7e868016bb867d6011872',
    'postcode': '2647',
    'region': ''}],
  'assigned_branch_key': '8a818ed76bb7e868016bb8443c6313cb',
  'birth_date': '1963-05-21',
  'client_role_key': '8a818f9f6bb76211016bb83eb1310d6d',
  'creation_date': '2019-07-03T08:15:41-07:00',
  'custom_field_sets': [{'custom_field_set_id': '_Food_Clients',
    'custom_field_values': [{'custom_field_id': 'Type_Clients',
      'custom_field_value': 'Chicken'}]},
   {'custom_field_set_id': '_Custom_Fields_Clients',
    'custom_field_values': [{'custom_field_id': 'Position_Clients',
      'custom_field_value': 'Agricultural product sorter 3'}]}],
  'encoded_key': '8a818ed76bb7e868016bb867d6011872',
  'first_name': 'Ela',
  'gender': 'FEMALE',
  'group_lo

In [149]:
params = {
    "detailsLevel": "FULL", 
    "offset": 0,
    "limit": 10,
    "paginationDetails": "ON"
}
endpoint = "deposits/transactions:search"
method = "POST"
body = {
    "sortingCriteria": {
    "field": "creationDate",
    "order": "ASC"
    },
    "filterCriteria": [
        {
            "field": "creationDate",
            "value": "2019-07-01",
            "operator": "AFTER"
        }
    ]
}

querystring = '&'.join(['%s=%s' % (key, value) for (key, value) in params.items()])

data, total_records = client.request(
    method=method,
    path=endpoint,
    params=querystring,
    endpoint=endpoint,
    json=body)

INFO METRIC: {"tags": {"status": "succeeded", "http_status_code": 200, "endpoint": "deposits/transactions:search"}, "type": "timer", "metric": "http_request_duration", "value": 0.7089946269989014}


In [150]:
data

[{'accountBalances': {'totalBalance': 75.0},
  'adjustmentTransactionKey': '8a8186e36c00c289016c01b8d82f35bb',
  'affectedAmounts': {'feesAmount': 0.0,
   'fractionAmount': 0.0,
   'fundsAmount': 75.0,
   'interestAmount': 0.0,
   'overdraftAmount': 0.0,
   'overdraftFeesAmount': 0.0,
   'overdraftInterestAmount': 0.0,
   'technicalOverdraftAmount': 0.0,
   'technicalOverdraftInterestAmount': 0.0},
  'amount': 75.0,
  'branchKey': '8a818ed76bb7e868016bb8443c6313cb',
  'creationDate': '2019-07-17T13:51:28-07:00',
  'currencyCode': 'USD',
  'encodedKey': '8a8187846bfc7fd7016c01b448124a70',
  'fees': [],
  'id': '1',
  'parentAccountKey': '8a8187846bfc7fd7016c014f6dc42710',
  'taxes': {},
  'terms': {'interestSettings': {'interestRate': 5.0},
   'overdraftInterestSettings': {},
   'overdraftSettings': {}},
  'transactionDetails': {'transactionChannelId': 'cash',
   'transactionChannelKey': '8a818f9f6bb76211016bb83eab1d0d69'},
  'transferDetails': {},
  'type': 'DEPOSIT',
  'userKey': '8a8

In [152]:
params = {
    "detailsLevel": "FULL", 
    "offset": 0,
    "limit": 10,
    "paginationDetails": "ON"
}
endpoint = "communications/messages:search"
method = "POST"
body = [{"field": "state", "operator": "EQUALS", "value": "SENT"}, {"field": "creationDate", "operator": "AFTER", "value": "2019-01-01"}]

querystring = '&'.join(['%s=%s' % (key, value) for (key, value) in params.items()])

data, total_records = client.request(
    method=method,
    path=endpoint,
    params=querystring,
    endpoint=endpoint,
    json=body)

INFO METRIC: {"tags": {"status": "succeeded", "http_status_code": 200, "endpoint": "communications/messages:search"}, "type": "timer", "metric": "http_request_duration", "value": 0.17499923706054688}
